In [1]:
import pandas as pd
from pathlib import Path
import requests
import json
from config import geoapify_key 
from config import api_key
import geopy
import numpy
from pathlib import Path

In [10]:
from tkinter import *

root=Tk()

root.title("Estimate Rates")
root.resizable(0,0)#Width, height
root.iconbitmap("Truck.ico")

rate = StringVar()

miframe = Frame()
miframe.pack(fill="both", expand="true")
miframe.config(width=650, height=350)

Label(miframe, text="Loading City").grid(row=0, column=0, sticky=W, pady=5, padx=10)
Label(miframe, text="Delivery City").grid(row=1, column=0, sticky=W, pady=5, padx=10)
Label(miframe, text="Estimate Rate").grid(row=2, column=0, sticky=W, pady=5, padx=10)

loadingentry = Entry(miframe)
loadingentry.grid(row=0, column=1, pady=4, padx=10)

deliveryentry = Entry(miframe)
deliveryentry.grid(row=1, column=1, pady=4, padx=10)

estimateentry = Entry(miframe, textvariable=rate)
estimateentry.grid(row=2, column=1, pady=4, padx=10)    

def codbutton():
    
    url_geo = f"https://api.geoapify.com/v1/geocode/search?format=json&apiKey={geoapify_key}&text="
    url_geo_miles = f"https://api.geoapify.com/v1/routing?mode=drive&units=imperial&apiKey={geoapify_key}&waypoints=" 

    responsecoorload = requests.get(url_geo + loadingentry.get()).json()
    lat_loading = str(responsecoorload['results'][0]['lat'])
    lon_loading = str(responsecoorload['results'][0]['lon'])
        
    responsecoordel = requests.get(url_geo + deliveryentry.get()).json()
    lat_drop = str(responsecoordel['results'][0]['lat'])
    lon_drop = str(responsecoordel['results'][0]['lon'])

    Coordloading = lat_loading + ',' + lon_loading
    Coorddrop = lat_drop + ',' + lon_drop

    dist = str(Coordloading) +'|'+ str(Coorddrop)

    response = requests.get(url_geo_miles + dist).json()
    miles = round(float(response['features'][0]['properties']['distance']),2)

    if(miles<=800):
        milesf = 800


    url = f"https://api.eia.gov/v2/petroleum/pri/gnd/data/?frequency=weekly&data[0]=value&facets[duoarea][]=R20&facets[product][]=EPD2D&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key}"
    response_url = requests.get(url).json()
    fuel = round(float(response_url['response']['data'][0]['value']),2)
    
    file_perc = Path("Fuelperc.csv")
    file_perc_df = pd.read_csv(file_perc)
    
    fuelper = file_perc_df[file_perc_df['value'] == fuel]
    fuelperfinal = fuelper['%'].sum()
    
    fuelfinal = str((milesf*1.8) + (milesf*fuelperfinal))

    rate.set(fuelfinal)

buttoncalculate = Button(root, text="Calculate", command=codbutton)
buttoncalculate.pack()

root.mainloop()